<a href="https://colab.research.google.com/github/BrotherKim/AI599/blob/master/finalproject/230626_drunken_statutes_prison_bin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KAIST AI599 법률인공지능

In [1]:
# train variables
TARGET = 'prison'
TARGET_SET = 'avail' # all, avail, nonavail. nonavail의 경우 법령이 파싱안되는 문제 발생.
TARGET_START = 0 # 0 if you want zero values, INCLUDE THST VALUE
TARGET_END = 36 # INCLUDE THIS VALUE
TARGET_APPROX = 0 # '2'개월 또는 '1000000'원
TARGET_OPT = 'binary' # class, binary
TRAIN_COLUMN = 'statutes' # raw, facts, statutes, yh, fy
MODEL_DIR = '/content/drive/MyDrive/AI599/models'

## 과제 개요

이름: 강여진, 김동규, 김학성, 김형, 여인경

개요
- 컬럼은 한글로 사용
- model은 영대문자 사용
- dataset은 영소문자, 숫자 사용


데이터 준비
- precedent는 주문을 형량으로 바꾼 것(이하 "형"), 양형의 사유(이하 "양"), 그리고 나머지(이하 "나")로 이루어져 있음. 양형의 사유와 나머지를 포함한 것을 "양나"라고 칭함.
- "형"을 벌금만 있는 경우와 징역만 있는 경우, 그리고 집행유예가 있는 징역의 경우를 추출. 각각 "벌", "징", "집"이라 칭함.
- 양형의 사유가 없는 집합에서 "벌"-"나", "징"-"나", "집"-"나" 데이터셋 생성. 이를 a, b, c라고 칭함
- 양형의 사유가 있는 집합에서 "벌"-"나", "징"-"나", "집"-"나", "벌"-"양나", "징"-"양나", "집"-"양나" 데이터셋 생성. 이를 d, e, f, g, h, i라고 칭함.
- a~i까지 데이터 생성 시, 해당하지 않는 라벨에 대해서도 예측하도록 함.


판결문에 대한 형량 예측 모델 학습
- 모델 A, B 준비. A에는 a로 "나"에 대한 "벌" 예측 모델 생성. B에는 b로 "나"에 대한 "징" 예측 모델 생성, C에는 c로 "나"에 대한 "집" 예측 모델 생성.


예측
- A에 d를 eval시켜 "나"에 해당하는 "벌" 예측. 이 결과쌍을 j라고 칭함.
- B에 e를 eval시켜 "나"에 해당하는 "징" 예측. 이 결과쌍을 k라고 칭함.
- C에 f를 eval시켜 "나"에 해당하는 "집" 예측. 이 결과쌍을 l라고 칭함.


양형 사유에 대한 형량 예측 모델 학습
- d, g의 "벌"의 delta 값을 추출. "양"-delta 데이터셋 생성. 이하 m.
- e, h의 "징"의 delta 값을 추출. "징"-delta 데이터셋 생성. 이하 n.
- f, i의 "집"의 delta 값을 추출. "집"-delta 데이터셋 생성. 이하 o.
- 모델 C, D, E 준비. m, n, o에 대해 학습.

## Google drive connect.
connect google drive storage for saving checkpoints.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# !rm -rf /content/drive/MyDrive/AI599/models
# !mkdir -p /content/drive/MyDrive/AI599/models

In [4]:
# # save models
# torch.save(models['prison'], '/content/drive/MyDrive/AI599/models//prison_model.pt')
# #torch.save(money_model, '/content/drive/MyDrive/AI599/models/money_model.pt')
# #torch.save(yooye_model, '/content/drive/MyDrive/AI599/models/yooye_model.pt')

## lbox_open data 전처리.
lbox_open 데이터를 가져와 형태를 분석합니다.

In [5]:
!pip install -q datasets==2.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.8 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.1/133.1 kB 19.8 MB/s eta 0:00:00


In [6]:
!pip install --quiet transformers==4.22.0
!pip install --quiet sentencepiece==0.1.97
!pip install --quiet pytorch-lightning==1.7.6
!pip install --quiet sklearn
!pip install --quiet torchsampler

# !pip install --quiet pytorch>=1.10.0
# !pip install --quiet transformers==4.16.2
# !pip install --quiet pytorch-lightning==1.5.10
!pip install --quiet streamlit==1.2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 67.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 707.5/707.5 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 52.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 62.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 71.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 26.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
import datasets

dataset_card = "lbox/lbox_open"
task = "precedent_corpus"
data = datasets.load_dataset(dataset_card, task)

Generating train split: 0 examples [00:00, ? examples/s]

Dataset lbox_open downloaded and prepared to /root/.cache/huggingface/datasets/lbox___lbox_open/precedent_corpus/0.2.0/3d5761e2d9292b674a2adabbe7f4bc200d1985908e6f657e7953c9bf247da7ae. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

데이터를 개별 확인해 보면, 아래와 같은 형태를 띄고 있는 것을 볼 수 있습니다.

In [8]:
data

DatasetDict({
    train: Dataset({
        features: ['id', 'precedent'],
        num_rows: 150000
    })
})

In [9]:
data_t = data["train"]
data_t[40]

{'id': 40,
 'precedent': "주문\n1. 제1심판결을 취소한다.\n2. 원고의 청구를 기각한다\n3. 소송비용은 제1, 2심을 합하여 원고의 부담으로 한다.\n\n청구취지및항소취지\n1. 청구취지\n피고는 원고에게 1,500,000원 및 이에 대하여 2004. 7. 9.부터 다 갚는 날까지 연 20%의 비율에 의한 돈을 지급하라.\n2. 항소취지\n주문과 같다.\n\n이유\n1. 기초사실\n다음 각 사실은 당사자 사이에 다툼이 없거나 갑 제1 내지 3호증, 을 제1호증의 각 기재에 변론의 전취지를 종합하여 이를 인정할 수 있다.\n가. 원고는 2004. 4. 29. 주식회사 세기메디칼에 엑스레이필름대금 150만 원을 송금하려고 하였으나 거래 상대방에 대한 착오를 일으켜, 원고가 거래하던 한국외환은행 주식회사(이하 '외환은행'이라 한다)에 인터넷 뱅킹을 통하여 원고의 예금 150만 원을 피고의 나운동 지점에 개설된 소외인의 계좌로 이체를 의뢰하였고, 외환은행은 즉시 150만 원을 피고 은행에 송금하였으며, 피고는 동시에 소외인의 계좌에 150만 원의 입금기장을 하였다.\n나. 소외인은 ○○인쇄소를 경영하면서 2003. 8.경까지 원고와 거래하였던 자인데, 2001. 11. 24. 피고로부터 500만 원을 대출받고는 2003. 12. 25. 이후 원리금 5,153,821원을 연체하였다.\n다. 피고는 2004. 5. 6. 위 대출금채권을 자동채권으로 하고 소외인의 위 150만 원에 대한 예금반환채권을 수동채권으로 하여 상계처리하였다.\n2. 주장 및 판단\n가. 청구원인\n원고는 소외인과는 거래가 없었는데 피고에 개설된 그의 계좌를 주식회사 세기메디칼의 것으로 착오하는 바람에 소외인의 계좌로 150만 원을 입금한 것이므로, 피고가 소외인의 계좌에 150만 원이 입금된 것을 기화로 자신의 그에 대한 대출금채권과 동액 상당을 상계한 것은 법률상 원인 없이 부당이득한 것이므로, 피고는 원고에게 150만 원을 반환해야 할 의무가 있다.\n나. 판 단\n

## filtering lbox_open data
2019.6.25 이후에 발생한 음주운전 데이터를 추출합니다.

In [10]:
import pandas as pd
df = pd.DataFrame( (v for v in data['train']) )
# df = pd.Series( (v['precedent'] for v in data['train']) )

In [11]:
df

,id,precedent
0,0,주문\n상고를 모두 기각한다.\n상고비용은 원고들의 부담으로 한다.\n\n이유\n상...
1,1,주문\n상고를 기각한다.\n상고비용은 피고의 부담으로 한다.\n\n이유\n상고이유를...
2,2,"주문\n원심판결을 파기하고, 사건을 서울고등법원에 환송한다.\n\n이유\n1. 피고..."
3,3,주문\n피고인의 항소를 기각한다.\n\n이유\n1. 항소이유의 요지\n피고인에 대한...
4,4,주문\n상고를 기각한다.\n상고 소송비용은 원고의 부담으로 한다.\n\n이유\n원고...
...,...,...
149995,149995,주문\n검사의 항소를 기각한다.\n\n이유\n1. 공소사실 및 원심의 판단\n가. ...
149996,149996,주문\n1. 제1심판결을 취소한다.\n2. 원고들의 청구를 모두 기각한다.\n3. ...
149997,149997,"주문\n원심판결중 임료상당 손해에 관한 피고들의 패소부분을 파기하고, 이 부분 사건..."
149998,149998,주문\n원심판결을 파기하고 사건을 서울고등법원에 환송한다.\n\n이유\n상고이유를 ...


### 주문이 없는 경우. 이유부터 시작하는데 알수가 없다 ㅋ

In [12]:
# 주문과 형량이 나타나는 사용 가능한 데이터 우선 추출
avail_df = df[ \
            ~df['precedent'].str.contains('주문') \
        #  &  df['precedent'].str.contains('법령의 적용|법령의적용') \
           #& df['precedent'].str.contains('피고인을') \
           #& df['precedent'].str.contains('처한다.') \
        #    & df['precedent'].str.contains('2020|2021') # 2019.6.25 개정 이후 사건만 다룸
        #    & df['precedent'].str.contains('도로교통법') \
        #    & df['precedent'].str.contains('제44조 제1항') \
        #    & df['precedent'].str.contains('범 죄 사 실|범죄사실') \
        #    & df['precedent'].str.contains('양형의 이유|양형의이유') \
           #& (df['precedent'].str.len() < 2500)
           ]

print(len(avail_df))

555


In [13]:
avail_df.loc[24].precedent

"이유\n제1,2점에 대하여 무릇 오인은 직업선택에 대한 자유를 형유하며 이 자유는 질서의 유지 또는 공공의 복리를 위하여 필요한 경우가 아니면 법률에 의하여서도 제한하지 못하는 것이 헌법의 정한 바이라 이 원칙은 직업의 종류에 따라서 다를배 없을 것인 바 불특정한 다수내객을 상대로 하는 소위 접객업에 있어서는 그 영업시설여하가 직접 공익에 미치는 영향이 지대하다 할지니 이에 행정상 취췌의 필요가 있다 할 것이다 그러한 고로 목욕탕영업에 있어서는 과거 일정시에 소위 경무국령으로 탕옥 영업취췌 규칙을 제정하여 탕옥시설의 규모와 업자의 업무상 주의사항을 규정하고 조건을 구비한 자는 영업장소 소재지 경찰서장에게 원출하여 허가를 수할 것을 명하였으니 동 규칙에 용어는 비록 허가라 하였으나 그 기본성질은 취췌해제에 불과한 것인즉 동 규칙 소요의 각 조건을 구비하여 원출하는 자에 대하여는 당해 경찰서장은 반드시 허가하여야 할 것으로 해석함이 동규칙 내지 헌법정신에 조감하여 타당하다 할 것이다 연이 왕시에 간간 허가관청 내지 그 감독기관에서 업자상호간의 이해조절을 위하여 업자의 원수 또는 장소 호상간의 거리를 제한하는 예규 내지 통첩등을 제정한 사례가 허다하였으며 본건에 있어서 단기1956년6월19일 내무부치안국장 명의로 각 시도 경찰국장에게 대하여 발한 '욕탕업소 상호거리 제한 변경에 관한 지시(갑 제6호증 참조)도 그 사례의 하나이다 연즉 그 지시는 서상의 법리로 보아서 경찰사무 취급에 대한 일종의 내부 훈시에 불과한 것으로 해석되므로 경찰서장이 목욕탕영업을 허가함에 제하여 전시 지시에 위배하였다 할지라도 동 허가행위를 즉시 위법이라 논단하지 못할 것이 명백한 바 원판결에 의하면 원심은 피고의 소외인에 대한 목욕탕 허가가 전시치안국장의 지시 명령 취지에 위배되는 것이라 하여 동 허가행위를 즉시 위법한 처분이라 인정하였으니 이는 서상의 법리를 간과한 소치라 할 것이다\n제3점에 대하여\n원판결에 의하면 원고 2는 피고의 본건 행정처분으로 인하여 기존 욕탕업자로서의 권리침

### 법령의 적용이 없는 경우. 기각|파기|취소|환송 문구가 있는 데이터가 105814개.
나머지 9998개는 원고와 피고간 민사로 보여짐. 자세히 봐야할 필요가 있음.

In [14]:
# 주문과 형량이 나타나는 사용 가능한 데이터 우선 추출
avail_df = df[ \
            df['precedent'].str.contains('주문') \
          & ~df['precedent'].str.contains('법령의 적용|법령의적용') \
          # 115812
          & ~df['precedent'].str.contains('기각|파기|취소|환송') \
           #& df['precedent'].str.contains('피고인을') \
           #& df['precedent'].str.contains('처한다.') \
        #    & df['precedent'].str.contains('2020|2021') # 2019.6.25 개정 이후 사건만 다룸
        #    & df['precedent'].str.contains('도로교통법') \
        #    & df['precedent'].str.contains('제44조 제1항') \
        #    & df['precedent'].str.contains('범 죄 사 실|범죄사실') \
        #    & df['precedent'].str.contains('양형의 이유|양형의이유') \
           #& (df['precedent'].str.len() < 2500)
           ]

print(len(avail_df))

9998


In [15]:
avail_df.precedent

10        주문\n1. 피고는 원고에게 별지 목록 기재 건물을 인도하라.\n2. 소송비용은 피...
25        주문\n1. 피고는 원고에게 37,000,000원을 지급하라.\n2. 소송비용은 피...
57        주문\n1. 별지 1. 기재 부동산을 경매하여 그 매각대금에서 경매비용을 공제한 나...
67        주문\n피고인은 무죄.\n\n이유\n1. 공소사실의 요지\n이 사건 공소사실의 요지...
69        주문\n1. 피고들은 원고에게,\n별지 목록 기재 각자의 상속지분표시에 의한,\n별...
                                ...                        
149954    주문\n피고인은 무죄.\n\n이유\n1. 공소사실의 요지\n피고인은 현재 토지임대업...
149956    주문\n1. 원고에게, 피고 B은 3,000,000원 및 이에 대하여 2019. 1...
149961    주문\n1. 피고는 원고에게 77,607,260원 및 이에 대하여 2021. 6. ...
149967    주문\n1. 피고 C는 D과 연대하여 원고에게 금 12,650,000원 및 이에 대...
149977    주문\n도시계획법(제정 1962.1.20. 법률 제983호, 최종개정 1991.12...
Name: precedent, Length: 9998, dtype: object

In [16]:
print(avail_df.loc[25].precedent)

주문
1. 피고는 원고에게 37,000,000원을 지급하라.
2. 소송비용은 피고가 부담한다.
3. 제1항은 가집행할 수 있다.

청구취지
주문과 같다.

이유
1. 청구의 표시
별지 청구원인 기재와 같다.
2. 공시송달에 따른 판결(민사소송법 제208조 제3항 제3호)


In [17]:
# 주문과 형량이 나타나는 사용 가능한 데이터 우선 추출
avail_df = df[ \
            df['precedent'].str.contains('주문') \
          & df['precedent'].str.contains('법령의 적용|법령의적용') \
           #& df['precedent'].str.contains('피고인을') \
           #& df['precedent'].str.contains('처한다.') \
        #    & df['precedent'].str.contains('2020|2021') # 2019.6.25 개정 이후 사건만 다룸
        #    & df['precedent'].str.contains('도로교통법') \
        #    & df['precedent'].str.contains('제44조 제1항') \
        #    & df['precedent'].str.contains('범 죄 사 실|범죄사실') \
        #    & df['precedent'].str.contains('양형의 이유|양형의이유') \
           #& (df['precedent'].str.len() < 2500)
           ]

print(len(avail_df))

33633


### 2020|2021, 도로교통법 제 44조 제1항에 대해선 모두 범죄자실이 존재한다.
파기라는 단어가 존재하는 데이터는 총 111개개

In [18]:
# 주문과 형량이 나타나는 사용 가능한 데이터 우선 추출
avail_df = df[ \
            df['precedent'].str.contains('주문') \
          & df['precedent'].str.contains('법령의 적용|법령의적용') \
           #& df['precedent'].str.contains('피고인을') \
           #& df['precedent'].str.contains('처한다.') \
           & df['precedent'].str.contains('2020|2021') # 2019.6.25 개정 이후 사건만 다룸
           & df['precedent'].str.contains('도로교통법') \
           & df['precedent'].str.contains('제44조 제1항') \
           & df['precedent'].str.contains('범 죄 사 실|범죄사실') \
           & ~df['precedent'].str.contains('양형의 이유|양형의이유') \
           #& (df['precedent'].str.len() < 2500)
           ]

print(len(avail_df))

579


In [19]:
# 주문과 형량이 나타나는 사용 가능한 데이터 우선 추출
avail_df = df[ \
            df['precedent'].str.contains('주문') \
          & df['precedent'].str.contains('법령의 적용|법령의적용') \
           #& df['precedent'].str.contains('피고인을') \
           #& df['precedent'].str.contains('처한다.') \
           & df['precedent'].str.contains('2020|2021') # 2019.6.25 개정 이후 사건만 다룸
           & df['precedent'].str.contains('도로교통법') \
           & df['precedent'].str.contains('제44조 제1항') \
           & df['precedent'].str.contains('범 죄 사 실|범죄사실') \
           & df['precedent'].str.contains('파기') \
        #    & df['precedent'].str.contains('양형의 이유|양형의이유') \
           #& (df['precedent'].str.len() < 2500)
           ]

print(len(avail_df))

111


In [20]:
# 주문과 형량이 나타나는 사용 가능한 데이터 우선 추출
avail_df = df[ \
            df['precedent'].str.contains('주문') \
          & df['precedent'].str.contains('법령의 적용|법령의적용') \
           #& df['precedent'].str.contains('피고인을') \
           #& df['precedent'].str.contains('처한다.') \
           & df['precedent'].str.contains('2020|2021') # 2019.6.25 개정 이후 사건만 다룸
           & df['precedent'].str.contains('도로교통법') \
           & df['precedent'].str.contains('제44조 제1항') \
           & df['precedent'].str.contains('범 죄 사 실|범죄사실') \
           & ~df['precedent'].str.contains('파기') \
        #    & df['precedent'].str.contains('양형의 이유|양형의이유') \
           #& (df['precedent'].str.len() < 2500)
           ]

print(len(avail_df))

3436


### 특이케이스 제외. ~에 대한 해당법조가 없는 경우 빼고, 피고인이 여럿인 경우 뺌.
위는 음주운전, 아래는 포맷에 맞는 전체 데이터수

In [21]:
# 주문과 형량이 나타나는 사용 가능한 데이터 우선 추출
avail_df = df[ \
            df['precedent'].str.contains('주문') \
          & df['precedent'].str.contains('법령의 적용|법령의적용') \
           #& df['precedent'].str.contains('피고인을') \
           #& df['precedent'].str.contains('처한다.') \
        #    & df['precedent'].str.contains('2020|2021') # 2019.6.25 개정 이후 사건만 다룸
        #    & df['precedent'].str.contains('도로교통법') \
        #    & df['precedent'].str.contains('제44조 제1항') \
           & df['precedent'].str.contains('범 죄 사 실|범죄사실') \
           & ~df['precedent'].str.contains('파기') \
           & ~df['precedent'].str.contains('피고인 A') \
        #    & df['precedent'].str.contains('양형의 이유|양형의이유') \
           #& (df['precedent'].str.len() < 2500)
           ]

print(len(avail_df))

25648


In [22]:
# 주문과 형량이 나타나는 사용 가능한 데이터 우선 추출
avail_df = df[ \
            df['precedent'].str.contains('주문') \
          & df['precedent'].str.contains('법령의 적용|법령의적용') \
           #& df['precedent'].str.contains('피고인을') \
           #& df['precedent'].str.contains('처한다.') \
           & df['precedent'].str.contains('2020|2021') # 2019.6.25 개정 이후 사건만 다룸
           & df['precedent'].str.contains('도로교통법') \
           & df['precedent'].str.contains('제44조 제1항') \
           & df['precedent'].str.contains('범 죄 사 실|범죄사실') \
           & ~df['precedent'].str.contains('파기') \
        #    & df['precedent'].str.contains('에 대한 해당') \
           & ~df['precedent'].str.contains('피고인 A') \
           # 인경님께 여쭤볼것. 법조항호목 순서를 안지킬수도 있는지? 152조는 다 호밖에 없네 ㅋ
           #& ~df['precedent'].str.contains('제152조 제1호') \
           & ~df['precedent'].str.contains('성매매알선 등 행위') \
        #    & df['precedent'].str.contains('양형의 이유|양형의이유') \
           #& (df['precedent'].str.len() < 2500)
           ]

print(len(avail_df))


3384


In [23]:
avail_df

,id,precedent
38,38,"주문\n피고인을 벌금 1,000만원에 처한다.\n피고인이 위 벌금을 납입하지 아니하..."
52,52,"주문\n피고인을 징역 1년 4개월에 처한다.\n다만, 이 판결 확정일부터 3년간 위..."
82,82,주문\n피고인을 징역 1년에 처한다.\n\n이유\n범 죄 사 실\n피고인은 2017...
145,145,"주문\n피고인을 벌금 1,000만 원에 처한다.\n피고인이 위 벌금을 납입하지 아니..."
205,205,"주문\n피고인을 징역 1년에 처한다.\n다만, 이 판결 확정일부터 2년간 위 형의 ..."
...,...,...
149879,149879,"주문\n피고인을 징역 1년 2월에 처한다.\n다만, 이 판결 확정일부터 3년간 위 ..."
149883,149883,"주문\n피고인을 벌금 10,000,000원에 처한다.\n피고인이 위 벌금을 납입하지..."
149890,149890,"주문\n피고인을 징역 8월에 처한다.\n다만, 이 판결 확정일부터 2년간 위 형의 ..."
149914,149914,"주문\n피고인을 징역 1년에 처한다.\n다만, 이 판결 확정일부터 2년간 위 형의 ..."


### 학습용 데이터로 변경.
주문, 범죄사실, 법령목록, 양형의이유로 쪼갬.

In [24]:
# 양형의 사유가 있는 것과 없는 것 분리
yesy = avail_df[avail_df['precedent'].str.contains('양형의 이유|양형의이유')]
noy = avail_df[~avail_df['precedent'].str.contains('양형의 이유|양형의이유')]
yesy

,id,precedent
52,52,"주문\n피고인을 징역 1년 4개월에 처한다.\n다만, 이 판결 확정일부터 3년간 위..."
82,82,주문\n피고인을 징역 1년에 처한다.\n\n이유\n범 죄 사 실\n피고인은 2017...
145,145,"주문\n피고인을 벌금 1,000만 원에 처한다.\n피고인이 위 벌금을 납입하지 아니..."
229,229,"주문\n피고인을 벌금 6,000,000원에 처한다.\n피고인이 위 벌금을 납입하지 ..."
240,240,"주문\n피고인을 징역 1년에 처한다.\n다만, 이 판결 확정일부터 2년간 위 형의 ..."
...,...,...
149879,149879,"주문\n피고인을 징역 1년 2월에 처한다.\n다만, 이 판결 확정일부터 3년간 위 ..."
149883,149883,"주문\n피고인을 벌금 10,000,000원에 처한다.\n피고인이 위 벌금을 납입하지..."
149890,149890,"주문\n피고인을 징역 8월에 처한다.\n다만, 이 판결 확정일부터 2년간 위 형의 ..."
149914,149914,"주문\n피고인을 징역 1년에 처한다.\n다만, 이 판결 확정일부터 2년간 위 형의 ..."


In [25]:
def get_leaf_nodes(laws, parentIdxMap):
    retval = []
    # 각 노드의 부모 인덱스를 저장하는 리스트 생성
    parents = [parentIdxMap[i] for i in range(len(laws))]

    # 자식 노드를 저장하는 리스트 생성
    children = [[] for _ in range(len(laws))]
    for i in range(len(laws)):
        parent = parents[i]
        children[parent].append(i)

    # 말단 노드를 찾아서 출력
    for i in range(len(laws)):
        if not children[i]:
            l = ''
            for lidx in get_ancestors(i, parents):
                l = '%s %s' % (l, laws[lidx]) if 0 < len(l) else laws[lidx]
            l = '%s %s' % (l, laws[i]) if 0 < len(l) else laws[i]
            retval.append(l)

    return retval

def get_ancestors(idx, parents):
    # idx 노드의 모든 조상 노드를 구해서 리스트에 추가
    ancestors = []
    parent = parents[idx]
    while parent != 0:
        ancestors.append(parent)
        parent = parents[parent]
    return list(reversed(ancestors))


def parse_law(text):
    laws = ['root']
    parentIdxMap = {0:0}
    lastIdxMap = {}

    for token in text.split(','):
        cur = token
        cur = cur.strip()
        ls = []
        while(0 < len(cur)):
            cur = cur.strip()
            # print(cur)
            if cur.endswith('항') or cur.endswith('호') or cur.endswith('목'):
                cur = '%s$' % (cur)
                ts = cur.split()
                l = ts[-1]
                cur = cur.replace(l, '')
                l = l[0:-1]
                # print('항호목: %s' % (l))
                ls.append(l)
            elif cur.endswith('법') or cur.endswith('률'):
                l = cur
                cur = ''
                # print('법률: %s' % (l))
                ls.append(l)
            else:
                cur = '%s$' % (cur)
                ts = cur.split()
                l = ts[-1]
                cur = cur.replace(l, '')
                l = l[0:-1]
                # print('조: %s' % (l))
                ls.append(l)
        for l in ls[::-1]:
            # print(l)
            # print(laws)
            # print(parentIdxMap)
            if l.endswith('목'):
                idx = len(laws)
                lastIdxMap[l[-1]] = idx
                parentIdxMap[idx] = lastIdxMap['호']
                laws.append(l)
            elif l.endswith('호'):
                idx = len(laws)
                lastIdxMap[l[-1]] = idx
                # print(laws)
                # print(parentIdxMap)
                # print(lastIdxMap)
                pidx = lastIdxMap['법']
                if '항' in lastIdxMap:
                    pidx = lastIdxMap['항']
                parentIdxMap[idx] = pidx
                laws.append(l)
            elif l.endswith('항'):
                idx = len(laws)
                lastIdxMap[l[-1]] = idx
                parentIdxMap[idx] = lastIdxMap['조']
                laws.append(l)
            elif l.endswith('법') or l.endswith('률'):
                idx = len(laws)
                lastIdxMap['법'] = idx
                parentIdxMap[idx] = 0
                laws.append(l)
            else:
                idx = len(laws)
                lastIdxMap['조'] = idx
                parentIdxMap[idx] = lastIdxMap['법']
                laws.append(l)

            # print('cur: %s\n' % (cur))

    return get_leaf_nodes(laws, parentIdxMap)

    # for l in laws[::-1]:
    #     print(l)
    # print(lastIdxMap)
    # return laws

sample_text = ",특정범죄 가중처벌 등에 관한 법률 제5조의3 제1항 제2호, 형법 제268조,,,형법 제37조 , 제38조 제1항 제2호, 제50조,,형법 제70조 제1항, 제69조 제2항"
parsed_laws = parse_law(sample_text)
#print(parsed_laws)



In [26]:
# Precedent 쪼개기 위한 function set
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

import re


def getKeys(d, prefix=''):
    result = []
    for key, value in d.items():
        current_prefix = f'{prefix} {key}'.strip()
        if value:
            result += getKeys(value, current_prefix)
        else:
            result.append(current_prefix)
    return result

class Precedent:
    def __init__(self, rawPrecedent, tokenizer, model):
        self.raw = rawPrecedent
        self.tokenizer = tokenizer
        self.model = model

        # text = """
        # 1일 오후 9시까지 최소 20만3220명이 코로나19에 신규 확진됐다. 또다시 동시간대 최다 기록으로, 사상 처음 20만명대에 진입했다.
        # 방역 당국과 서울시 등 각 지방자치단체에 따르면 이날 0시부터 오후 9시까지 전국 신규 확진자는 총 20만3220명으로 집계됐다.
        # 국내 신규 확진자 수가 20만명대를 넘어선 것은 이번이 처음이다.
        # 동시간대 최다 기록은 지난 23일 오후 9시 기준 16만1389명이었는데, 이를 무려 4만1831명이나 웃돌았다. 전날 같은 시간 기록한 13만3481명보다도 6만9739명 많다.
        # 확진자 폭증은 3시간 전인 오후 6시 집계에서도 예견됐다.
        # 오후 6시까지 최소 17만8603명이 신규 확진돼 동시간대 최다 기록(24일 13만8419명)을 갈아치운 데 이어 이미 직전 0시 기준 역대 최다 기록도 넘어섰다. 역대 최다 기록은 지난 23일 0시 기준 17만1451명이었다.
        # 17개 지자체별로 보면 서울 4만6938명, 경기 6만7322명, 인천 1만985명 등 수도권이 12만5245명으로 전체의 61.6%를 차지했다. 서울과 경기는 모두 동시간대 기준 최다로, 처음으로 각각 4만명과 6만명을 넘어섰다.
        # 비수도권에서는 7만7975명(38.3%)이 발생했다. 제주를 제외한 나머지 지역에서 모두 동시간대 최다를 새로 썼다.
        # 부산 1만890명, 경남 9909명, 대구 6900명, 경북 6977명, 충남 5900명, 대전 5292명, 전북 5150명, 울산 5141명, 광주 5130명, 전남 4996명, 강원 4932명, 충북 3845명, 제주 1513명, 세종 1400명이다.
        # 집계를 마감하는 자정까지 시간이 남아있는 만큼 2일 0시 기준으로 발표될 신규 확진자 수는 이보다 더 늘어날 수 있다. 이에 따라 최종 집계되는 확진자 수는 21만명 안팎을 기록할 수 있을 전망이다.
        # 한편 전날 하루 선별진료소에서 이뤄진 검사는 70만8763건으로 검사 양성률은 40.5%다. 양성률이 40%를 넘은 것은 이번이 처음이다. 확산세가 계속 거세질 수 있다는 얘기다.
        # 이날 0시 기준 신규 확진자는 13만8993명이었다. 이틀 연속 13만명대를 이어갔다.
        # """

        # text = text.replace('\n', ' ')

        # raw_input_ids = self.tokenizer.encode(text)
        # input_ids = [self.tokenizer.bos_token_id] + raw_input_ids + [self.tokenizer.eos_token_id]

        # summary_ids = self.model.generate(torch.tensor([input_ids]),  num_beams=4,  max_length=512,  eos_token_id=1)
        # result = self.tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)

        # result

        self.__parse__()

    def __getStatutesList__(self, statRaw):
        # laws 딕셔너리를 statutesList로 변환
        #statutesList = getKeys(laws)
        # print('-'*100)
        # print(statRaw)
        statutesList = parse_law(statRaw)
        # print(statutesList)
        # print('-'*100)

        return statutesList


    def __parse__(self):
        # 주문 - 범죄사실 - 법령의적용 - 양형의이유
        # ms - facts - statuts - yh
        # fy: facts + yh
        #self.raw = self.raw.replace('\n', ' ')
        self.etc = self.raw

        self.etc = self.etc.replace('범 죄 사 실', '범죄사실')
        self.etc = self.etc.replace('법령의 적용', '법령의적용')
        self.etc = self.etc.replace('양형의 이유', '양형의이유')

        tag = '범죄사실'
        l = self.etc.split(tag)
        self.ms = ''
        if 1 < len(l):
            self.ms = l[0].strip()
            self.etc = self.etc.replace(self.ms, '')
            self.etc = self.etc.replace(tag, '')
            self.etc = self.etc.strip()
            self.ms = self.ms.replace('\n', '')

        tag = '법령의적용'
        self.facts = ''
        l = self.etc.split(tag)
        if 1 < len(l):
            self.facts = l[0].strip()
            self.etc = self.etc.replace(self.facts, '')
            self.etc = self.etc.replace(tag, '')
            self.etc = self.etc.strip()
            self.facts = self.facts.replace('\n', '')

        tag = '양형의이유'
        self.statutes = ''
        l = self.etc.split(tag)
        if 1 < len(l):
            self.statutes = l[0].strip()
            self.etc = self.etc.replace(self.statutes, '')

            self.statutes = re.sub('[123]\.\s*[^\d]+\\n', '\\n', self.statutes)
            self.statutes = re.sub('[123]\.\s*[^\d]+:', '', self.statutes)
            self.statutes = self.statutes.replace('\n', ',')
            self.statutes = re.sub('[\[\(\{}].+[\]\)\}]', '', self.statutes)
            self.statutes = self.statutes.replace('(아래', '')
            self.statutes = self.statutes.replace('전단', '')
            self.statutes = self.statutes.replace('각 ', '')
            self.statutes = self.statutes.replace('구 ', '')
            self.statutes = self.statutes.replace('(', '')
            self.statutes = self.statutes.replace('징역형 선택', '')
            self.statutes = self.statutes.replace('금고형 선택', '')
            self.statutes = self.statutes.replace('벌금형 선택', '')
            self.statutes = self.statutes.replace('○ ', '')
            self.statutes = self.statutes.replace('판시 제1의 점', '')
            self.statutes = self.statutes.replace('도로교통법죄에 정한 형으로 처벌', '')
            self.statutes = self.statutes.replace('「', '')
            self.statutes = self.statutes.replace('」', '')
            self.statutes = self.statutes.replace('특정범죄 가중처벌 등에 관한 법률죄에 대하여는 벌금형을 선택', '')
            self.statutes = self.statutes.replace('법률제', '법률 제')

            self.etc = self.etc.replace(tag, '')
            #self.etc = self.etc.strip()
            sl = self.__getStatutesList__(self.statutes)
            sl.sort()
            if len(sl) == 0:
                print(self.statutes)
                print(sl)

            self.statutes = ', '.join(sl)
            #print(self.statutes)
        else:
            print('len == 0')
            print(l)

        self.yh = self.etc
        self.etc = self.raw

        # print(f'before facts:\n {self.facts}')
        # raw_input_ids = self.tokenizer.encode(self.facts)
        # input_ids = [self.tokenizer.bos_token_id] + raw_input_ids + [self.tokenizer.eos_token_id]
        # summary_ids = self.model.generate(torch.tensor([input_ids]),  num_beams=4,  max_length=512,  eos_token_id=1)
        # self.facts = self.tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)
        # print(f'after facts:\n {self.facts}')

        self.fy = f'{self.facts} {self.yh}'


def getNumericInKorean(kStrNum):
    retval = 1
    retStr = kStrNum
    retStr = retStr.replace(',', '')
    if '만' in retStr:
        retval = retval * 10000
        retStr = retStr.replace('만', '')
    if '천' in retStr:
        retval = retval * 1000
        retStr = retStr.replace('천', '')
    retval = int(retStr) * retval
    return retval

def getMonthFromText(s):
    year  = re.findall(r'(\d+)년', s, re.S)
    y = 0 if len(year) == 0 else (int)(year[0]);
    month  = re.findall(r'(\d+)개?월', s, re.S)
    m = 0 if len(month) == 0 else (int)(month[0]);
    return y*12 + m

def generateDataSet(dataframe, max_cnt, classopt='class'):
    retval = pd.DataFrame({\
                           'id': pd.Series(dtype='int'), 'money': pd.Series(dtype='int'), 'prison': pd.Series(dtype='int'),'yooye': pd.Series(dtype='int'),\
                           'facts':[], 'statutes':[], 'yh':[], 'fy':[], 'raw':[]})

    tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')
    model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')

    cnts = {}
    for i, row in dataframe.iterrows():
        p = Precedent(row['precedent'], tokenizer, model)
        #print(f'p.ms: {p.ms}, p.fy: {p.fy}')
        # if '처한다' in p.ms and p.ms.startswith('피') and '및' not in p.ms:
        info = {}
        info['money'] = 0
        info['prison'] = 0
        info['yooye'] = 0
        # 주문\n피고인을 벌금 14,000,000(일천사백만)원에 처한다.\
        money  = re.findall(r'벌금\s+([^\s\(\)]+)\s*원', p.ms, re.S)
        if 0 < len(money): # 벌금형
            if classopt == 'class':
                info['money'] = '0' if len(money) == 0 else getNumericInKorean(money[0])
            else:
                info['money'] = '0' if len(money) == 0 else 1
        else: # 금고형, 집행유예 포함
            if '유예' in p.ms: # 집행유예 포함된 금고형.
                msList = p.ms.split('처한다')
                info['yooye'] = getMonthFromText(msList[1]) if classopt == 'class' else 1
                info['prison'] = getMonthFromText(msList[0]) if classopt == 'class' else 1
                # yy = 1
                #prison = 1
            else: # 그냥 금고형
                info['prison'] = getMonthFromText(p.ms) if classopt == 'class' else 1
                #prison = 1

        if TARGET_START <= info[TARGET] and info[TARGET] <= TARGET_END and info[TARGET] not in []:
        #    if prison in [0, 12, 18]:
            if info[TARGET] not in cnts:
                cnts[info[TARGET]] = 0
            cnts[info[TARGET]] = cnts[info[TARGET]] + 1
            if cnts[info[TARGET]] > max_cnt:
                continue
            #retval.loc[i] = [i, [info['money']], [info['prison']], [info['yooye']], p.facts, p.statutes, p.yh, p.fy, p.raw]
            retval.loc[i] = [i, info['money'], info['prison'], info['yooye'], p.facts, p.statutes, p.yh, p.fy, p.raw]

        #retval.loc[i] = [i, [mn], [prison], [yy], p.etc, p.raw]
    return retval

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [27]:
# 양형의 사유가 없는 집합에서 "벌"-"나", "징"-"나" 데이터셋 생성. 이를 a, b라고 칭함
dataset = {}
dataset['all'] = generateDataSet(avail_df, 100000, TARGET_OPT)
dataset['avail'] = generateDataSet(yesy, 100000, TARGET_OPT)
dataset['nonavail'] = generateDataSet(noy, 100000, TARGET_OPT)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


len == 0
['1. 에 대한 해당법조 및 형의 선택\n구 도로교통법(2020. 6. 9. 법률 제17371호로 개정되기 전의 것) 제148조의2 제1항, 제44조 제1항(벌금형 선택)\n1. 노역장유치\n형법 제70조 제1항, 제69조 제2항\n1. 가납명령\n형사소송법 제334조 제1항']
len == 0
['1. 에 대한 해당법조 및 형의 선택\n도로교통법 제148조의2 제1항, 제44조 제1항, 징역형 선택\n1. 작량감경\n형법 제53조, 제55조 제1항 제3호\n1. 집행유예\n형법 제62조 제1항(피고인이 자신의 잘못을 인정하고 반성하면서 재범하지 않을 것을 다짐하는 점 등을 참작)']
len == 0
['1. 에 대한 해당법조 및 형의 선택\n도로교통법 제148조의2 제3항 제1호, 제44조 제1항, 벌금형 선택(혈중알콜농도 수치가 매우 높고, 이 사건 음주운전으로 교통사고가 야기되기는 하였으나, 피고인이 이 사건 범행을 깊이 반성하고 있는 점, 위 교통사고는 물적 피해에 그친 점, 피고인은 차량을 매도한 점, 징역형을 선택할 경우 직장을 잃을 위험이 있는 점, 피고인은 아무런 처벌 전력이 없는 점 등을 참작)\n1. 노역장유치\n형법 제70조 제1항, 제69조 제2항\n1. 가납명령\n형사소송법 제334조 제1항\n이상의 이유로 주문과 같이 판결한다.']
len == 0
['1. 에 대한 해당법조\n도로교통법 제148조의2 제3항 제2호, 제44조 제1항\n1. 형의 선택\n징역형 선택(피고인의 음주운전 전력 등 고려)\n1. 집행유예\n형법 제62조 제1항(현재 피고인이 자신의 잘못을 깊이 뉘우치면서 다시는 음주운전을 하지 않겠다고 다짐하고 있는 점 등 참작)\n1. 사회봉사명령\n형법 제62조의2']
len == 0
['1. 에 대한 해당법조\n도로교통법 제148조의2 제3항 제2호, 제44조 제1항(징역형 선택)\n1. 집행유예\n형법 제62조 제1항(피고인이 과거 음주운전으로 거듭 처벌받은 전력이 있기는 하나, 잘못을 뉘우치고 있는 

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


len == 0
['1. 에 대한 해당법조 및 형의 선택\n구 도로교통법(2020. 6. 9. 법률 제17371호로 개정되기 전의 것) 제148조의2 제1항, 제44조 제1항(벌금형 선택)\n1. 노역장유치\n형법 제70조 제1항, 제69조 제2항\n1. 가납명령\n형사소송법 제334조 제1항']
len == 0
['1. 에 대한 해당법조 및 형의 선택\n도로교통법 제148조의2 제1항, 제44조 제1항, 징역형 선택\n1. 작량감경\n형법 제53조, 제55조 제1항 제3호\n1. 집행유예\n형법 제62조 제1항(피고인이 자신의 잘못을 인정하고 반성하면서 재범하지 않을 것을 다짐하는 점 등을 참작)']
len == 0
['1. 에 대한 해당법조 및 형의 선택\n도로교통법 제148조의2 제3항 제1호, 제44조 제1항, 벌금형 선택(혈중알콜농도 수치가 매우 높고, 이 사건 음주운전으로 교통사고가 야기되기는 하였으나, 피고인이 이 사건 범행을 깊이 반성하고 있는 점, 위 교통사고는 물적 피해에 그친 점, 피고인은 차량을 매도한 점, 징역형을 선택할 경우 직장을 잃을 위험이 있는 점, 피고인은 아무런 처벌 전력이 없는 점 등을 참작)\n1. 노역장유치\n형법 제70조 제1항, 제69조 제2항\n1. 가납명령\n형사소송법 제334조 제1항\n이상의 이유로 주문과 같이 판결한다.']
len == 0
['1. 에 대한 해당법조\n도로교통법 제148조의2 제3항 제2호, 제44조 제1항\n1. 형의 선택\n징역형 선택(피고인의 음주운전 전력 등 고려)\n1. 집행유예\n형법 제62조 제1항(현재 피고인이 자신의 잘못을 깊이 뉘우치면서 다시는 음주운전을 하지 않겠다고 다짐하고 있는 점 등 참작)\n1. 사회봉사명령\n형법 제62조의2']
len == 0
['1. 에 대한 해당법조\n도로교통법 제148조의2 제3항 제2호, 제44조 제1항(징역형 선택)\n1. 집행유예\n형법 제62조 제1항(피고인이 과거 음주운전으로 거듭 처벌받은 전력이 있기는 하나, 잘못을 뉘우치고 있는 

In [28]:
dataset['all'].statutes
#yesy = avail_df[avail_df['precedent'].str.contains('양형의 이유|양형의이유')]

38                                                         
52        특정범죄 가중처벌 등에 관한 법률 제5조의11 제1항, 형법 제37조, 형법 제38...
82        도로교통법 제148조의2 제1항, 도로교통법 제44조 제1항, 형법 제53조, 형법...
145       특정범죄 가중처벌 등에 관한 법률 전문, 특정범죄 가중처벌 등에 관한 법률 제5조의...
205                                                        
                                ...                        
149879    도로교통법 제148조의2 제1항, 도로교통법 제44조 제1항, 형법 제53조, 형법...
149883    도로교통법 제148조의2 제1항, 도로교통법 제44조 제1항, 형법 제69조 제2항...
149890    도로교통법 제148조의2 제3항 제2호, 도로교통법 제44조 제1항, 형법 제53조...
149914    도로교통법 제148조의2 제3항 제1호, 도로교통법 제44조 제1항, 형법 제53조...
149937    특정범죄 가중처벌 등에 관한 법률 제5조의11 제1항, 형법 제37조, 형법 제38...
Name: statutes, Length: 3384, dtype: object

In [29]:
dataset['avail']

,id,money,prison,yooye,facts,statutes,yh,fy,raw
52,52,0,1,1,1. 특정범죄가중처벌등에관한법률위반(위험운전치상)피고인은 B 말리부 승용차의 운전업...,"특정범죄 가중처벌 등에 관한 법률 제5조의11 제1항, 형법 제37조, 형법 제38...",\n\n1. 양형기준에 따른 권고형의 범위\n[유형의 결정] 교통범죄 > 02. 위...,1. 특정범죄가중처벌등에관한법률위반(위험운전치상)피고인은 B 말리부 승용차의 운전업...,"주문\n피고인을 징역 1년 4개월에 처한다.\n다만, 이 판결 확정일부터 3년간 위..."
82,82,0,1,0,피고인은 2017. 8. 18. 창원지방법원에서 도로교통법위반(음주운전)죄 등으로 ...,"도로교통법 제148조의2 제1항, 도로교통법 제44조 제1항, 형법 제53조, 형법...","\n\n아래의 정상 및 그 밖에 피고인의 연령, 직업, 성행, 환경, 범행의 동기 ...",피고인은 2017. 8. 18. 창원지방법원에서 도로교통법위반(음주운전)죄 등으로 ...,주문\n피고인을 징역 1년에 처한다.\n\n이유\n범 죄 사 실\n피고인은 2017...
145,145,1,0,0,1. 특정범죄가중처벌등에관한법률위반(위험운전치상)피고인은 B 아이오닉 승용차의 운전...,"특정범죄 가중처벌 등에 관한 법률 전문, 특정범죄 가중처벌 등에 관한 법률 제5조의...",\n\n술에 취하여 차량을 운행하고 교통사고를 일으켜 피해자들에게 상해를 입게 한 ...,1. 특정범죄가중처벌등에관한법률위반(위험운전치상)피고인은 B 아이오닉 승용차의 운전...,"주문\n피고인을 벌금 1,000만 원에 처한다.\n피고인이 위 벌금을 납입하지 아니..."
229,229,1,0,0,1. 도로교통법위반(음주운전)피고인은 2020. 11. 1. 18:45경 충주시 B...,"도로교통법 제148조의2 제3항 제2호, 도로교통법 제44조 제1항, 형법 제37조...","\n\n아래의 정상 및 피고인의 나이, 성행, 가정환경, 범행의 동기, 수단과 결과...",1. 도로교통법위반(음주운전)피고인은 2020. 11. 1. 18:45경 충주시 B...,"주문\n피고인을 벌금 6,000,000원에 처한다.\n피고인이 위 벌금을 납입하지 ..."
240,240,0,1,1,피고인은 2020. 5. 29. 창원지방법원 거창지원에서 도로교통법위반(음주운전)죄...,"도로교통법, 형법 제37조, 형법 제38조 제1항 제2호, 형법 제50조, 형법 제...",\n\n1. 법률상 처단형의 범위: 징역 1년 ~ 3년\n2. 선고형의 결정: 징역...,피고인은 2020. 5. 29. 창원지방법원 거창지원에서 도로교통법위반(음주운전)죄...,"주문\n피고인을 징역 1년에 처한다.\n다만, 이 판결 확정일부터 2년간 위 형의 ..."
...,...,...,...,...,...,...,...,...,...
149879,149879,0,1,1,피고인은 2010. 12. 23. 대전지방법원 서산지원에서 도로교통법위반(음주운전)...,"도로교통법 제148조의2 제1항, 도로교통법 제44조 제1항, 형법 제53조, 형법...","\n\n혈중알콜농도가 매우 높은 점, 3회의 동종전력과 다수의 이종 범죄전력 및 단...",피고인은 2010. 12. 23. 대전지방법원 서산지원에서 도로교통법위반(음주운전)...,"주문\n피고인을 징역 1년 2월에 처한다.\n다만, 이 판결 확정일부터 3년간 위 ..."
149883,149883,1,0,0,피고인은 2011. 11. 10. 서울중앙지방법원에서 도로교통법위반(음주운전)죄로 ...,"도로교통법 제148조의2 제1항, 도로교통법 제44조 제1항, 형법 제69조 제2항...","\n\n피고인이 범행을 인정하고 반성하는 점, 음주운전의 위험성과 처벌 필요성, 피...",피고인은 2011. 11. 10. 서울중앙지방법원에서 도로교통법위반(음주운전)죄로 ...,"주문\n피고인을 벌금 10,000,000원에 처한다.\n피고인이 위 벌금을 납입하지..."
149890,149890,0,1,1,피고인은 2021. 6. 26. 00:09경 부산 부산진구 중앙대로 730에 있는 ...,"도로교통법 제148조의2 제3항 제2호, 도로교통법 제44조 제1항, 형법 제53조...","\n\n벌금형을 초과하는 형사처벌을 받은 전력이 없는 점, 자신의 잘못을 반성하고 ...",피고인은 2021. 6. 26. 00:09경 부산 부산진구 중앙대로 730에 있는 ...,"주문\n피고인을 징역 8월에 처한다.\n다만, 이 판결 확정일부터 2년간 위 형의 ..."
149914,149914,0,1,1,피고인은 2021. 7. 1. 22:24경 수원시 장안구 연무동 이하 불상지에서부터...,"도로교통법 제148조의2 제3항 제1호, 도로교통법 제44조 제1항, 형법 제53조...","\n\n음주운전은 운전자 자신뿐만 아니라 불특정 다수의 생명, 신체를 해할 수 있는...",피고인은 2021. 7. 1. 22:24경 수원시 장안구 연무동 이하 불상지에서부터...,"주문\n피고인을 징역 1년에 처한다.\n다만, 이 판결 확정일부터 2년간 위 형의 ..."


In [30]:
dataset['all'][TARGET].value_counts()

1    2647
0     737
Name: prison, dtype: int64

In [31]:
dataset['avail'][TARGET].value_counts()

1    2300
0     538
Name: prison, dtype: int64

In [32]:
dataset['nonavail'][TARGET].value_counts()

1    347
0    199
Name: prison, dtype: int64

In [33]:
# from imblearn.over_sampling import SMOTENC
# from sklearn.preprocessing import MultiLabelBinarizer

# # for reproducibility purposes
# seed = 100
# # SMOTE number of neighbors
# k = 1

# df = dataset['all']

In [34]:
# import torch
# from imblearn.over_sampling import RandomOverSampler

# from sklearn.model_selection import train_test_split
# import datasets
# import tensorflow as tf
# from datasets import Dataset

# import os
# os.environ['TOKENIZERS_PARALLELISM'] = 'false'

# from functools import reduce
# import random

# import torch
# import numpy as np
# import transformers
# import pytorch_lightning as pl
# from transformers import AutoTokenizer, BertForSequenceClassification

# import pandas as pd

# def convertDFtoDS(df):
#     return tf.data.Dataset.from_tensor_slices(dict(df))

# backbone_card = 'bert-base-multilingual-cased'
# tokenizer = AutoTokenizer.from_pretrained(backbone_card)


# def resampling_data(df):
#     inputs = tokenizer(df[TARGET].tolist(), return_tensors="pt", padding=True, truncation=True, max_length=512)
#     input_ids = inputs.input_ids
#     attention_mask = inputs.attention_mask
#     # (# of samples, # of features) 크기의 array, dataframe, sparse matrix 모두 가능
#     # 여기선 그냥 array
#     x = [[input_id, mask] for input_id, mask in zip(input_ids, attention_mask)]
#     y = df[TARGET].tolist()
#     return x, y

# ros = RandomOverSampler(random_state=42)

# x, y = resampling_data(df)
# x_ros, y_ros = ros.fit_resample(x, y)

# print(x)


# # class ResampledDataset(torch.utils.data.Dataset):
# #     def __init__(self, x_rus, y_rus):
# #         self.input_ids = []
# #         self.attention_mask = []
# #         for input_id, mask in x_rus:
# #             self.input_ids.append(input_id)
# #             self.attention_mask.append(mask)
# #         self.labels = y_rus

# #     def __len__(self):
# #         return len(self.input_ids)

# #     def __getitem__(self, idx):
# #         return {'input_ids': self.input_ids[idx] ,'attention_mask': self.attention_mask[idx], 'labels':self.labels[idx]}

# # # huggingface Trainer에 전달할 수 있는 Dataset
# # train_data = ResampledDataset(x_ros, y_ros)

# # ...

# # trainer = Trainer(
# #    model=model,
# #    args=training_args,
# #    compute_metrics=compute_metrics,
# #    train_dataset=train_data,
# #    eval_dataset=eval_data
# #    )


In [35]:
# # make a new df made of all the columns, except the target class
# X = df.loc[:, df.columns != TARGET]
# y = df[TARGET]
# #sm = SMOTE(sampling_strategy='auto', k_neighbors=k, random_state=seed)
# sm = SMOTENC(categorical_features=[0,], random_state=seed)
# X_res, y_res = sm.fit_resample(X, y)

In [36]:
# sm

In [37]:
# from imblearn.over_sampling import SMOTE
# sm = SMOTE(random_state = 42)
# X_train_oversampled, y_train_oversampled = sm.fit_sample(X_train, y_train)
# X_train = pd.DataFrame(X_train_oversampled, columns=X_train.columns)

In [38]:
# # hs's rule
# [1,2,3,4,5]
# [6,10]
# []
# []
# []

# # yk's rule
# [1 2 3]
# [4 5 6]
# [7 8 9] 8
# [10 11 12]
# [13 14 15] 14
# [16 17 18]

# # 1~5 / 6~8 / 9~12 / 13~15 / 16~18 / 19~22 / 24~29 / 30~36 /

## 전처리된 데이터로 학습.
모델이 학습할 수 있도록 데이터의 형태를 변경해 학습합니다.

In [39]:
import os
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

from functools import reduce
import random

import torch
import numpy as np
import transformers
import pytorch_lightning as pl
from transformers import AutoTokenizer, BertForSequenceClassification

import pandas as pd

In [40]:
# https://data-newbie.tistory.com/721
from torch.utils.data import Sampler
class OverSampler(Sampler):
    """Over Sampling
    Provides equal representation of target classes in each batch
    """
    def __init__(self, class_vector, batch_size):
        """
        Arguments
        ---------
        class_vector : torch tensor
            a vector of class labels
        batch_size : integer
            batch_size
        """
        self.n_splits = int(class_vector.size(0) / batch_size)
        self.class_vector = class_vector
        self.indices = list(range(len(self.class_vector)))
        self.batch_size =batch_size
        uni_label = torch.unique(class_vector)
        uni_label_sorted, _  = torch.sort(uni_label)
        print(uni_label_sorted)
        uni_label_sorted = uni_label_sorted.detach().numpy()
        label_bin = torch.bincount(class_vector.int()).detach().numpy()
        label_to_count = dict(zip(uni_label_sorted , label_bin))
        weights = [ len(class_vector) / label_to_count[float(label)] for label in           class_vector]
        self.weights = torch.DoubleTensor(weights)

    def __iter__(self):
        return (self.indices[i] for i in torch.multinomial(
            self.weights, self.batch_size, replacement=True))

    def __len__(self):
        return len(self.class_vector)

In [41]:
#trainsampler = OverSampler(class_vector=torch.from_numpy(label.values.squeeze()), batch_size=batch_size)
#trainloader = DataLoader(traindataset, batch_size=batch_size, shuffle=False,


In [42]:
dataset['avail']

,id,money,prison,yooye,facts,statutes,yh,fy,raw
52,52,0,1,1,1. 특정범죄가중처벌등에관한법률위반(위험운전치상)피고인은 B 말리부 승용차의 운전업...,"특정범죄 가중처벌 등에 관한 법률 제5조의11 제1항, 형법 제37조, 형법 제38...",\n\n1. 양형기준에 따른 권고형의 범위\n[유형의 결정] 교통범죄 > 02. 위...,1. 특정범죄가중처벌등에관한법률위반(위험운전치상)피고인은 B 말리부 승용차의 운전업...,"주문\n피고인을 징역 1년 4개월에 처한다.\n다만, 이 판결 확정일부터 3년간 위..."
82,82,0,1,0,피고인은 2017. 8. 18. 창원지방법원에서 도로교통법위반(음주운전)죄 등으로 ...,"도로교통법 제148조의2 제1항, 도로교통법 제44조 제1항, 형법 제53조, 형법...","\n\n아래의 정상 및 그 밖에 피고인의 연령, 직업, 성행, 환경, 범행의 동기 ...",피고인은 2017. 8. 18. 창원지방법원에서 도로교통법위반(음주운전)죄 등으로 ...,주문\n피고인을 징역 1년에 처한다.\n\n이유\n범 죄 사 실\n피고인은 2017...
145,145,1,0,0,1. 특정범죄가중처벌등에관한법률위반(위험운전치상)피고인은 B 아이오닉 승용차의 운전...,"특정범죄 가중처벌 등에 관한 법률 전문, 특정범죄 가중처벌 등에 관한 법률 제5조의...",\n\n술에 취하여 차량을 운행하고 교통사고를 일으켜 피해자들에게 상해를 입게 한 ...,1. 특정범죄가중처벌등에관한법률위반(위험운전치상)피고인은 B 아이오닉 승용차의 운전...,"주문\n피고인을 벌금 1,000만 원에 처한다.\n피고인이 위 벌금을 납입하지 아니..."
229,229,1,0,0,1. 도로교통법위반(음주운전)피고인은 2020. 11. 1. 18:45경 충주시 B...,"도로교통법 제148조의2 제3항 제2호, 도로교통법 제44조 제1항, 형법 제37조...","\n\n아래의 정상 및 피고인의 나이, 성행, 가정환경, 범행의 동기, 수단과 결과...",1. 도로교통법위반(음주운전)피고인은 2020. 11. 1. 18:45경 충주시 B...,"주문\n피고인을 벌금 6,000,000원에 처한다.\n피고인이 위 벌금을 납입하지 ..."
240,240,0,1,1,피고인은 2020. 5. 29. 창원지방법원 거창지원에서 도로교통법위반(음주운전)죄...,"도로교통법, 형법 제37조, 형법 제38조 제1항 제2호, 형법 제50조, 형법 제...",\n\n1. 법률상 처단형의 범위: 징역 1년 ~ 3년\n2. 선고형의 결정: 징역...,피고인은 2020. 5. 29. 창원지방법원 거창지원에서 도로교통법위반(음주운전)죄...,"주문\n피고인을 징역 1년에 처한다.\n다만, 이 판결 확정일부터 2년간 위 형의 ..."
...,...,...,...,...,...,...,...,...,...
149879,149879,0,1,1,피고인은 2010. 12. 23. 대전지방법원 서산지원에서 도로교통법위반(음주운전)...,"도로교통법 제148조의2 제1항, 도로교통법 제44조 제1항, 형법 제53조, 형법...","\n\n혈중알콜농도가 매우 높은 점, 3회의 동종전력과 다수의 이종 범죄전력 및 단...",피고인은 2010. 12. 23. 대전지방법원 서산지원에서 도로교통법위반(음주운전)...,"주문\n피고인을 징역 1년 2월에 처한다.\n다만, 이 판결 확정일부터 3년간 위 ..."
149883,149883,1,0,0,피고인은 2011. 11. 10. 서울중앙지방법원에서 도로교통법위반(음주운전)죄로 ...,"도로교통법 제148조의2 제1항, 도로교통법 제44조 제1항, 형법 제69조 제2항...","\n\n피고인이 범행을 인정하고 반성하는 점, 음주운전의 위험성과 처벌 필요성, 피...",피고인은 2011. 11. 10. 서울중앙지방법원에서 도로교통법위반(음주운전)죄로 ...,"주문\n피고인을 벌금 10,000,000원에 처한다.\n피고인이 위 벌금을 납입하지..."
149890,149890,0,1,1,피고인은 2021. 6. 26. 00:09경 부산 부산진구 중앙대로 730에 있는 ...,"도로교통법 제148조의2 제3항 제2호, 도로교통법 제44조 제1항, 형법 제53조...","\n\n벌금형을 초과하는 형사처벌을 받은 전력이 없는 점, 자신의 잘못을 반성하고 ...",피고인은 2021. 6. 26. 00:09경 부산 부산진구 중앙대로 730에 있는 ...,"주문\n피고인을 징역 8월에 처한다.\n다만, 이 판결 확정일부터 2년간 위 형의 ..."
149914,149914,0,1,1,피고인은 2021. 7. 1. 22:24경 수원시 장안구 연무동 이하 불상지에서부터...,"도로교통법 제148조의2 제3항 제1호, 도로교통법 제44조 제1항, 형법 제53조...","\n\n음주운전은 운전자 자신뿐만 아니라 불특정 다수의 생명, 신체를 해할 수 있는...",피고인은 2021. 7. 1. 22:24경 수원시 장안구 연무동 이하 불상지에서부터...,"주문\n피고인을 징역 1년에 처한다.\n다만, 이 판결 확정일부터 2년간 위 형의 ..."


In [43]:
ccnt = dataset[TARGET_SET][TARGET].value_counts()

class_counts = dataset[TARGET_SET][TARGET].value_counts().to_list() #43200, 4800
num_samples = sum(class_counts) # 48000 - 전체 데이터 갯수
labels = dataset[TARGET_SET][TARGET].to_list()

ccnt_map = {}
for index, row in dataset[TARGET_SET][TARGET].value_counts().iteritems():
    ccnt_map[index] = (float)(num_samples/row)

weights = []
for l in labels:
    weights.append(ccnt_map[l])

weights

<ipython-input-43-ef3fb0cd39fc>:8: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, row in dataset[TARGET_SET][TARGET].value_counts().iteritems():


[1.2339130434782608,
 1.2339130434782608,
 5.275092936802974,
 5.275092936802974,
 1.2339130434782608,
 1.2339130434782608,
 1.2339130434782608,
 1.2339130434782608,
 1.2339130434782608,
 5.275092936802974,
 1.2339130434782608,
 1.2339130434782608,
 1.2339130434782608,
 1.2339130434782608,
 1.2339130434782608,
 1.2339130434782608,
 1.2339130434782608,
 1.2339130434782608,
 1.2339130434782608,
 1.2339130434782608,
 1.2339130434782608,
 1.2339130434782608,
 1.2339130434782608,
 1.2339130434782608,
 5.275092936802974,
 1.2339130434782608,
 1.2339130434782608,
 1.2339130434782608,
 1.2339130434782608,
 1.2339130434782608,
 1.2339130434782608,
 1.2339130434782608,
 1.2339130434782608,
 5.275092936802974,
 1.2339130434782608,
 1.2339130434782608,
 1.2339130434782608,
 1.2339130434782608,
 1.2339130434782608,
 1.2339130434782608,
 5.275092936802974,
 5.275092936802974,
 1.2339130434782608,
 1.2339130434782608,
 1.2339130434782608,
 5.275092936802974,
 1.2339130434782608,
 1.2339130434782608,


In [44]:
len(weights)

2838

In [45]:
# # dataset[TARGET_SET][TARGET].value_counts().to_list()
# # dataset[TARGET_SET][TARGET].to_list()


# #class 0 : 43200개, class 1 : 4800개
# ccnt_map = dataset[TARGET_SET][TARGET].value_counts()
# class_counts = dataset[TARGET_SET][TARGET].value_counts().to_list() #43200, 4800
# num_samples = sum(class_counts) # 48000 - 전체 데이터 갯수
# labels = dataset[TARGET_SET][TARGET].to_list()

# #클래스별 가중치 부여 [48000/43200, 48000/4800] => class 1에 가중치 높게 부여하게 됨
# class_weights = [num_samples / class_counts[i] for i in range(len(class_counts))]
# print(labels)
# print(class_weights)
# # 해당 데이터의 label에 해당되는 가중치
# weights = [class_weights[labels[i]]
#            for i in \
#            range(int(num_samples))] #해당 레이블마다의 가중치 비율²_

In [46]:
# # https://rueki.tistory.com/244
# from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

# ###############################################
# #위에 데이터셋 class, 및 등등 선언되어있다 가정#
# ###############################################

# #class 0 : 43200개, class 1 : 4800개
# class_counts = y_train.value_counts().to_list() #43200, 4800
# num_samples = sum(class_counts) # 48000 - 전체 데이터 갯수
# labels = y_train.to_list()

# #클래스별 가중치 부여 [48000/43200, 48000/4800] => class 1에 가중치 높게 부여하게 됨
# class_weights = [num_samples / class_counts[i] for i in range(len(class_counts))]

# # 해당 데이터의 label에 해당되는 가중치
# weights = [class_weights[labels[i]] for i in range(int(num_samples))] #해당 레이블마다의 가중치 비율
# sampler = WeightedRandomSampler(torch.DoubleTensor(weights), int(num_samples))



In [47]:
test_d = None

In [48]:
#from torchsampler import ImbalancedDatasetSampler

from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

class StatutesDataModule(pl.LightningDataModule):
    def __init__(self, column, trainset, statutes, tokenizer, datadf, batch_size=16, max_input_len=512):

        super().__init__()
        self.column = column
        self.trainset = trainset
        self.statutes = statutes
        self.tokenizer = tokenizer
        #self.data = data
        self.batch_size = batch_size
        self.max_input_len = max_input_len

        #divide train/valid/test set
        train, test = train_test_split(datadf, test_size=0.2, random_state=1)
        train, valid = train_test_split(train, test_size=0.2, random_state=1)
        self.data = datasets.DatasetDict({'train': Dataset.from_dict(train), 'valid': Dataset.from_dict(valid), 'test': Dataset.from_dict(test)})

        ###############################################
        #위에 데이터셋 class, 및 등등 선언되어있다 가정#
        ###############################################

        #class 0 : 43200개, class 1 : 4800개
        ccnt = train[TARGET].value_counts()

        class_counts = train[TARGET].value_counts().to_list() #43200, 4800
        num_samples = sum(class_counts) # 48000 - 전체 데이터 갯수
        labels = train[TARGET].to_list()

        #클래스별 가중치 부여 [48000/43200, 48000/4800] => class 1에 가중치 높게 부여하게 됨
        ccnt_map = {}
        for index, row in train[TARGET].value_counts().iteritems():
            ccnt_map[index] = (float)(num_samples/row)

        # 해당 데이터의 label에 해당되는 가중치
        weights = []
        for l in labels:
            weights.append(ccnt_map[l])

        self.trainsampler = WeightedRandomSampler(torch.DoubleTensor(weights), int(num_samples)*1)

    def setup(self, stage):
        pass

    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.data['train'], batch_size=self.batch_size, shuffle=False, collate_fn=self._collate_fn, num_workers=2, sampler=self.trainsampler)

    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.data['valid'], batch_size=self.batch_size*2, shuffle=False, collate_fn=self._collate_fn)

    def test_dataloader(self):
        return torch.utils.data.DataLoader(self.data['test'], batch_size=self.batch_size*2, shuffle=False, collate_fn=self._collate_fn)

    def _collate_fn(self, batch):
        inputs = self.tokenizer([x[self.trainset] for x in batch], max_length=self.max_input_len, padding=True, truncation=True, return_tensors='pt')
        labels = [x[self.column] for x in batch]

        return inputs, labels

In [50]:
from sklearn.model_selection import train_test_split
import datasets
import tensorflow as tf
from datasets import Dataset

def convertDFtoDS(df):
    return tf.data.Dataset.from_tensor_slices(dict(df))

backbone_card = 'bert-base-multilingual-cased'
tokenizer = AutoTokenizer.from_pretrained(backbone_card)

In [58]:
class StatutesClassifier(pl.LightningModule):
    def __init__(self, statutes, backbone, learning_rate=1e-5, mode='multi-class', approx_val=2):
        super().__init__()
        self.statutes = statutes
        self.backbone = backbone
        self.learning_rate = learning_rate
        self.mode = mode
        self.criterion = torch.nn.CrossEntropyLoss() if mode == 'multi-class' else torch.nn.BCEWithLogitsLoss()
        self.approx_val = approx_val
        #self.criterion = torch.nn.MSELoss()
        #self.criterion = torch.nn.BCEWithLogitsLoss()

    def forward(self, batch):
        inputs, labels = batch
        logits = self.backbone(**inputs).logits
        targets = torch.zeros_like(logits)
        for i, label in enumerate(labels):
            label_id = self.statutes.index(label) if label < len(self.statutes) else 0
            # label_id = [(self.statutes.index(x) if x in self.statutes else 0) for x in label]
            #targets[i, label_id] = 1 / len(label_id) if self.mode == 'multi-class' else 1
            targets[i, label_id] = 1
        loss = self.criterion(logits, targets)

        return loss, logits

    def training_step(self, batch, batch_idx):
        loss, _ = self.forward(batch)
        return {'loss': loss}

    def validation_step(self, batch, batch_idx):
        return self._evaluation_step(batch)

    def validation_epoch_end(self, outputs):
        self._evaluation_epoch_end(outputs)

    def test_step(self, batch, batch_idx):
        return self._evaluation_step(batch)

    def test_epoch_end(self, outputs):
        self._evaluation_epoch_end(outputs, True)

    def _evaluation_step(self, batch):
        loss, logits = self.forward(batch)
        _, gts = batch
        if self.mode == 'multi-class':
            prs = np.array(self.statutes)[logits.argmax(-1).cpu()].tolist()
        else:
            prs = [np.array(self.statutes)[np.where(logit.sigmoid().cpu() > 0.5)[0]] for logit in logits]

        # if isinstance(prs, list) == False:
        #     prs = [prs]

        return {'loss': loss.item(), 'gts': gts, 'prs': prs}

    def _evaluation_epoch_end(self, outputs, test=False):
        avg_loss = np.mean([x['loss'] for x in outputs])
        print(outputs)
        gts = reduce(lambda x,y: x + y, [x['gts'] for x in outputs], [])
        prs = reduce(lambda x,y: x + y, [x['prs'] for x in outputs], [])
        if self.mode == 'multi-class':
            #acc = sum([(pr in gt) for gt, pr in zip(gts, prs)]) / len(gts)
            acc = sum([abs(pr - gt) <= self.approx_val for gt, pr in zip(gts, prs)]) / len(gts)
            # acc = sum([abs(pr - gt[0]) <= self.approx_val for gt, pr in zip(gts, prs)]) / len(gts)

        else:
            acc = sum([(set(pr) == set(gt)) for gt, pr in zip(gts, prs)]) / len(gts)

        print('='*50)
        print(f'avg_loss: {avg_loss}')
        print(f'ACC: {acc}')

        #target_ids = random.sample(range(len(gts)), 2)
        # target_ids = random.sample(range(len(prs)), 2)
        # print(gts)
        # print(prs)
        # print(f'target_ids: {target_ids}')
        # for target_id in target_ids:
        #     print(f'GT: {gts[target_id]}\t\t\tPR: {prs[target_id]}')

        # if self.mode == 'multi-class' and test:
        #     #df = pd.DataFrame(columns=['idx', 'gt', 'pr', 'etc'])
        #     #df = pd.DataFrame({'idx': pd.Series(dtype='int'), 'gt':pd.Series(dtype='int'), 'pr':pd.Series(dtype='int'),'etc':[]})
        #     df = pd.DataFrame({'idx': pd.Series(dtype='int'), 'gt':pd.Series(dtype='int'), 'pr':pd.Series(dtype='int')})
        #     print(data)
        #     for i, (gt, pr) in enumerate(zip(gts, prs)):
        #         if pr not in gt:
        #             # df = df.append(pd.DataFrame({'idx': [i],
        #             #                              'gt': ', '.join((str)(gt)),
        #             #                              'pr': pr,
        #             #                              'etc': data['test'][i]['etc']}),
        #             #                 ignore_index=True)
        #             df = df.append(pd.DataFrame({'idx': [i],
        #                                          'gt': ', '.join((str)(gt)),
        #                                          'pr': pr}),
        #                             ignore_index=True)
        #     df.to_csv('GT_vs_PR.csv', index=False)
    def configure_optimizers(self):
        grouped_params = [{'params': list(filter(lambda p: p.requires_grad, self.parameters())), 'lr': self.learning_rate}]
        optimizer = torch.optim.AdamW(grouped_params, lr=self.learning_rate)
 #       optimizer = torch.optim.AdamW(self.parameters(), lr=lr, weight_decay=weight_decay)
        scheduler = torch.optim.lr_scheduler.OneCycleLR(
                                max_lr=self.learning_rate,
                                epochs=20,
                                optimizer=optimizer,
                                steps_per_epoch=int(1480 / 16),
                                pct_start=0.1,
                                div_factor=10,
                                final_div_factor=100,
                                base_momentum=0.90,
                                max_momentum=0.95,
                                )
        # return [optimizer], [scheduler]
        return optimizer

#    def configure_optimizers(self):
#        grouped_params = [{'params': list(filter(lambda p: p.requires_grad, self.parameters())), 'lr': self.learning_rate}]
#        optimizer = torch.optim.AdamW(grouped_params, lr=self.learning_rate)
#
#        return {'optimizer': optimizer}

In [59]:
# # coded by cat
# cfdxf,er?"
# ]\w
# ws"

In [60]:
import gc

def execTrain(dataset, target, trainset):
    #init cuda
    gc.collect()
    torch.cuda.empty_cache()

    # #divide train/valid/test set
    # train, test = train_test_split(dataset, test_size=0.2, random_state=1)
    # train, valid = train_test_split(train, test_size=0.2, random_state=1)
    # d = datasets.DatasetDict({'train': Dataset.from_dict(train), 'valid': Dataset.from_dict(valid), 'test': Dataset.from_dict(test)})

    #print(set(dataset[target].to_list()))

    # labels init

    labels = sorted(set(dataset[target].to_list()))
    data_module = StatutesDataModule(target, trainset, labels, tokenizer, dataset, batch_size=16, max_input_len=512)
    #data_module = StatutesDataModule(target, trainset, labels, tokenizer, dataset, batch_size=16, max_input_len=384)

    # model
    backbone = BertForSequenceClassification.from_pretrained(backbone_card, num_labels=len(labels))
    model = StatutesClassifier(labels, backbone, learning_rate=2e-5, mode='multi-class', approx_val=TARGET_APPROX)

    # trainer
    n_gpus = 1 #torch.cuda.device_count()
    trainer = pl.Trainer(max_epochs=20, gpus=n_gpus, fast_dev_run=not True)

    # train and eval
    trainer.fit(model, data_module)
    trainer.test(model, data_module)

    return model

def execEval(dataset, target, trainset, modelpath):
    #init cuda
    gc.collect()
    torch.cuda.empty_cache()

    #divide train/valid/test set
    train, test = train_test_split(dataset, test_size=0.2, random_state=1)
    train, valid = train_test_split(train, test_size=0.2, random_state=1)
    d = datasets.DatasetDict({'train': Dataset.from_dict(train), 'valid': Dataset.from_dict(valid), 'test': Dataset.from_dict(test)})

    # labels init
    labels = sorted(set.union(*map(set, dataset[target])))
    data_module = StatutesDataModule(target, trainset, labels, tokenizer, d, batch_size=16, max_input_len=512)

    # model
    backbone = BertForSequenceClassification.from_pretrained(backbone_card, num_labels=len(labels))
    model = StatutesClassifier(labels, backbone, learning_rate=2e-5, mode='multi-class')
    model = torch.load(modelpath)

    # trainer
    n_gpus = 1 #torch.cuda.device_count()
    trainer = pl.Trainer(max_epochs=20, gpus=n_gpus, fast_dev_run=not True)

    trainer.test(model, data_module)

## RUN

In [61]:
# check variables
print(TARGET)
print(TARGET_SET)
print(TARGET_START)
print(TARGET_END)
print(TARGET_APPROX)
print(TARGET_OPT)
print(TRAIN_COLUMN)

prison
avail
0
36
0
binary
statutes


In [62]:
dataset[TARGET_SET]

,id,money,prison,yooye,facts,statutes,yh,fy,raw
52,52,0,1,1,1. 특정범죄가중처벌등에관한법률위반(위험운전치상)피고인은 B 말리부 승용차의 운전업...,"특정범죄 가중처벌 등에 관한 법률 제5조의11 제1항, 형법 제37조, 형법 제38...",\n\n1. 양형기준에 따른 권고형의 범위\n[유형의 결정] 교통범죄 > 02. 위...,1. 특정범죄가중처벌등에관한법률위반(위험운전치상)피고인은 B 말리부 승용차의 운전업...,"주문\n피고인을 징역 1년 4개월에 처한다.\n다만, 이 판결 확정일부터 3년간 위..."
82,82,0,1,0,피고인은 2017. 8. 18. 창원지방법원에서 도로교통법위반(음주운전)죄 등으로 ...,"도로교통법 제148조의2 제1항, 도로교통법 제44조 제1항, 형법 제53조, 형법...","\n\n아래의 정상 및 그 밖에 피고인의 연령, 직업, 성행, 환경, 범행의 동기 ...",피고인은 2017. 8. 18. 창원지방법원에서 도로교통법위반(음주운전)죄 등으로 ...,주문\n피고인을 징역 1년에 처한다.\n\n이유\n범 죄 사 실\n피고인은 2017...
145,145,1,0,0,1. 특정범죄가중처벌등에관한법률위반(위험운전치상)피고인은 B 아이오닉 승용차의 운전...,"특정범죄 가중처벌 등에 관한 법률 전문, 특정범죄 가중처벌 등에 관한 법률 제5조의...",\n\n술에 취하여 차량을 운행하고 교통사고를 일으켜 피해자들에게 상해를 입게 한 ...,1. 특정범죄가중처벌등에관한법률위반(위험운전치상)피고인은 B 아이오닉 승용차의 운전...,"주문\n피고인을 벌금 1,000만 원에 처한다.\n피고인이 위 벌금을 납입하지 아니..."
229,229,1,0,0,1. 도로교통법위반(음주운전)피고인은 2020. 11. 1. 18:45경 충주시 B...,"도로교통법 제148조의2 제3항 제2호, 도로교통법 제44조 제1항, 형법 제37조...","\n\n아래의 정상 및 피고인의 나이, 성행, 가정환경, 범행의 동기, 수단과 결과...",1. 도로교통법위반(음주운전)피고인은 2020. 11. 1. 18:45경 충주시 B...,"주문\n피고인을 벌금 6,000,000원에 처한다.\n피고인이 위 벌금을 납입하지 ..."
240,240,0,1,1,피고인은 2020. 5. 29. 창원지방법원 거창지원에서 도로교통법위반(음주운전)죄...,"도로교통법, 형법 제37조, 형법 제38조 제1항 제2호, 형법 제50조, 형법 제...",\n\n1. 법률상 처단형의 범위: 징역 1년 ~ 3년\n2. 선고형의 결정: 징역...,피고인은 2020. 5. 29. 창원지방법원 거창지원에서 도로교통법위반(음주운전)죄...,"주문\n피고인을 징역 1년에 처한다.\n다만, 이 판결 확정일부터 2년간 위 형의 ..."
...,...,...,...,...,...,...,...,...,...
149879,149879,0,1,1,피고인은 2010. 12. 23. 대전지방법원 서산지원에서 도로교통법위반(음주운전)...,"도로교통법 제148조의2 제1항, 도로교통법 제44조 제1항, 형법 제53조, 형법...","\n\n혈중알콜농도가 매우 높은 점, 3회의 동종전력과 다수의 이종 범죄전력 및 단...",피고인은 2010. 12. 23. 대전지방법원 서산지원에서 도로교통법위반(음주운전)...,"주문\n피고인을 징역 1년 2월에 처한다.\n다만, 이 판결 확정일부터 3년간 위 ..."
149883,149883,1,0,0,피고인은 2011. 11. 10. 서울중앙지방법원에서 도로교통법위반(음주운전)죄로 ...,"도로교통법 제148조의2 제1항, 도로교통법 제44조 제1항, 형법 제69조 제2항...","\n\n피고인이 범행을 인정하고 반성하는 점, 음주운전의 위험성과 처벌 필요성, 피...",피고인은 2011. 11. 10. 서울중앙지방법원에서 도로교통법위반(음주운전)죄로 ...,"주문\n피고인을 벌금 10,000,000원에 처한다.\n피고인이 위 벌금을 납입하지..."
149890,149890,0,1,1,피고인은 2021. 6. 26. 00:09경 부산 부산진구 중앙대로 730에 있는 ...,"도로교통법 제148조의2 제3항 제2호, 도로교통법 제44조 제1항, 형법 제53조...","\n\n벌금형을 초과하는 형사처벌을 받은 전력이 없는 점, 자신의 잘못을 반성하고 ...",피고인은 2021. 6. 26. 00:09경 부산 부산진구 중앙대로 730에 있는 ...,"주문\n피고인을 징역 8월에 처한다.\n다만, 이 판결 확정일부터 2년간 위 형의 ..."
149914,149914,0,1,1,피고인은 2021. 7. 1. 22:24경 수원시 장안구 연무동 이하 불상지에서부터...,"도로교통법 제148조의2 제3항 제1호, 도로교통법 제44조 제1항, 형법 제53조...","\n\n음주운전은 운전자 자신뿐만 아니라 불특정 다수의 생명, 신체를 해할 수 있는...",피고인은 2021. 7. 1. 22:24경 수원시 장안구 연무동 이하 불상지에서부터...,"주문\n피고인을 징역 1년에 처한다.\n다만, 이 판결 확정일부터 2년간 위 형의 ..."


In [63]:
type(dataset['avail'])

pandas.core.frame.DataFrame

In [64]:
# 양형의 이유 있는 데이터 중 etc로 prison classification, MSE
model = execTrain(dataset[TARGET_SET], TARGET, TRAIN_COLUMN)
torch.save(model, f'{MODEL_DIR}/{TARGET}_{TARGET_SET}_{TRAIN_COLUMN}_{TARGET_START}_{TARGET_END}_model_imb.pt')

<ipython-input-48-db2bc99a3741>:35: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, row in train[TARGET].value_counts().iteritems():
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be e

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:236: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


[{'loss': 0.5967882871627808, 'gts': [1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'prs': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}, {'loss': 0.6269877552986145, 'gts': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1], 'prs': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}]
avg_loss: 0.6118880212306976
ACC: 0.78125


/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:236: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


...

Validation: 0it [00:00, ?it/s]

[{'loss': 0.048672884702682495, 'gts': [1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'prs': [1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}, {'loss': 0.16438691318035126, 'gts': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1], 'prs': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1]}, {'loss': 0.034238606691360474, 'gts': [1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1], 'prs': [1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1]}, {'loss': 0.05192875117063522, 'gts': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1], 'prs': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1]}, {'loss': 0.015475381165742874, 'g

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:236: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

[{'loss': 0.2227269858121872, 'gts': [1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0], 'prs': [1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]}, {'loss': 0.20492783188819885, 'gts': [1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0], 'prs': [0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0]}, {'loss': 0.030172310769557953, 'gts': [0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'prs': [0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}, {'loss': 0.08507159352302551, 'gts': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1], 'prs': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1]}, {'loss': 0.020818691700696945, 'gts